In [1]:
#https://data101.oopy.io/recommendation-engine-cosine-similarity-naver-version-code-sharing

import pandas as pd
import numpy as np

df = pd.read_csv('shops.csv', sep=',')  

columns = ['상호명', '상권업종대분류명','상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '도로명주소', 
           '경도', '위도']
df=df[columns].copy()

df = df.loc[(df['시군구명'] == '서대문구')]  #서대문구에 있는 음식점만

df_ = df.loc[df.상권업종대분류명 =="음식"]  #음식점만

dong=list(sorted(set(list((df_['행정동명'])))))
len(dong)
dong

dong_list=[]
for i in range(len(dong)): 
    dong_list.append(df_.loc[df.행정동명==dong[i]])
    i+=1
dong_list[0]

,상호명,상권업종대분류명,상권업종소분류명,시도명,시군구명,행정동명,도로명주소,경도,위도
2634,감자바우곤드레밥,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로2길 7,126.915967,37.569179
7709,단비,음식,호프/맥주,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로 42,126.914240,37.570433
8090,고봉민김밥인,음식,라면김밥분식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917828,37.574368
14688,광풍정육식당,음식,갈비/삼겹살,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 14-4,126.914553,37.570671
20094,엄마맛집,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 6,126.914896,37.570475
...,...,...,...,...,...,...,...,...,...
302393,오늘은달달,음식,아이스크림판매,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917872,37.574422
303066,피자픽,음식,피자전문,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917872,37.574422
309103,커스텀커피DMC,음식,커피전문점/카페/다방,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917872,37.574422
312565,속초종합회어시장,음식,횟집,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로2길 14,126.916246,37.569443


In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

chromedriver = 'C:/Users/ChaeWon/crawling/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver) 
dong_list[0]['naver_keyword'] = dong_list[0]['행정동명']+'%20'+dong_list[0]['상호명']
dong_list[0]['naver_map_url'] = ''

# url 가져오기

for i, keyword in enumerate(dong_list[0]['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {dong_list[0].shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        driver.get(naver_map_search_url)
        
        time.sleep(3.5)
        dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')

        #검색결과 없는 경우
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):
            try:
                dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                dong_list[0].iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# url 만들어주기
dong_list[0]['naver_map_url'] = "https://m.place.naver.com/restaurant/" + dong_list[0]['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
dong_list[0] = dong_list[0].loc[~dong_list[0]['naver_map_url'].isnull()]

dong_list[0]

CH = dong_list[0]

CH.to_csv("충현동.csv", mode='w')

<ipython-input-3-3740e2e0aafa>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
<ipython-input-3-3740e2e0aafa>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_keyword'] = dong_list[0]['행정동명']+'%20'+dong_list[0]['상호명']
<ipython-input-3-3740e2e0aafa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_map_url'] = ''


이번에 찾을 키워드 : 0 / 115 행 남가좌1동%20감자바우곤드레밥


<ipython-input-3-3740e2e0aafa>:25: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
C:\Users\ChaeWon\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


이번에 찾을 키워드 : 1 / 115 행 남가좌1동%20단비
이번에 찾을 키워드 : 2 / 115 행 남가좌1동%20고봉민김밥인
이번에 찾을 키워드 : 3 / 115 행 남가좌1동%20광풍정육식당
이번에 찾을 키워드 : 4 / 115 행 남가좌1동%20엄마맛집
이번에 찾을 키워드 : 5 / 115 행 남가좌1동%20안뜰


<ipython-input-3-3740e2e0aafa>:31: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D49943+2595139]
	Ordinal0 [0x00CDC9F1+2148849]
	Ordinal0 [0x00BD4528+1066280]
	Ordinal0 [0x00C00FD4+1249236]
	Ordinal0 [0x00C011CB+1249739]
	Ordinal0 [0x00C2D812+1431570]
	Ordinal0 [0x00C1BA34+1358388]
	Ordinal0 [0x00C2BAF2+1424114]
	Ordinal0 [0x00C1B806+1357830]
	Ordinal0 [0x00BF6086+1204358]
	Ordinal0 [0x00BF6F96+1208214]
	GetHandleVerifier [0x00EEB232+1658114]
	GetHandleVerifier [0x00FA312C+2411516]
	GetHandleVerifier [0x00DDF261+560433]
	GetHandleVerifier [0x00DDE366+556598]
	Ordinal0 [0x00CE286B+2173035]
	Ordinal0 [0x00CE75F8+2192888]
	Ordinal0 [0x00CE76E5+2193125]
	Ordinal0 [0x00CF11FC+2232828]
	BaseThreadInitThunk [0x76496739+25]
	RtlGetFullPathName_UEx [0x77DA8E7F+1215]
	RtlGetFullPathName_UEx

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D49943+2595139]
	Ordinal0 [0x00CDC9F1+2148849]
	Ordinal0 [0x00BD4528+1066280]
	Ordinal0 [0x00C00FD4+1249236]
	Ordinal0 [0x00C011CB+1249739]
	Ordinal0 [0x00C2D812+1431570]
	Ordinal0 [0x00C1BA34+1358388]
	Ordinal0 [0x00C2BAF2+1424114]
	Ordinal0 [0x00C1B806+1357830]
	Ordinal0 [0x00BF6086+1204358]
	Ordinal0 [0x00BF6F96+1208214]
	GetHandleVerifier [0x00EEB232+1658114]
	GetHandleVerifier [0x00FA312C+2411516]
	GetHandleVerifier [0x00DDF261+560433]
	GetHandleVerifier [0x00DDE366+556598]
	Ordinal0 [0x00CE286B+2173035]
	Ordinal0 [0x00CE75F8+2192888]
	Ordinal0 [0x00CE76E5+2193125]
	Ordinal0 [0x00CF11FC+2232828]
	BaseThreadInitThunk [0x76496739+25]
	RtlGetFullPathName_UEx [0x77DA8E7F+1215]
	RtlGetFullPathName_UEx

이번에 찾을 키워드 : 63 / 115 행 남가좌1동%20모닥불
이번에 찾을 키워드 : 64 / 115 행 남가좌1동%20선인장
이번에 찾을 키워드 : 65 / 115 행 남가좌1동%20이디야커피
이번에 찾을 키워드 : 66 / 115 행 남가좌1동%20일단네식당
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D49943+2595139]
	Ordinal0 [0x00CDC9F1+2148849]
	Ordinal0 [0x00BD4528+1066280]
	Ordinal0 [0x00C00FD4+1249236]
	Ordinal0 [0x00C011CB+1249739]
	Ordinal0 [0x00C2D812+1431570]
	Ordinal0 [0x00C1BA34+1358388]
	Ordinal0 [0x00C2BAF2+1424114]
	Ordinal0 [0x00C1B806+1357830]
	Ordinal0 [0x00BF6086+1204358]
	Ordinal0 [0x00BF6F96+1208214]
	GetHandleVerifier [0x00EEB232+1658114]
	GetHandleVerifier [0x00FA312C+2411516]
	GetHandleVerifier [0x00DDF261+560433]
	GetHandleVerifier [0x00DDE366+556598]
	Ordinal0 [0x00CE286B+2173035]
	Ordinal0 [0x00CE75F8+2192888]
	Ordinal0 [0x00C

이번에 찾을 키워드 : 111 / 115 행 남가좌1동%20오늘은달달
이번에 찾을 키워드 : 112 / 115 행 남가좌1동%20피자픽
이번에 찾을 키워드 : 113 / 115 행 남가좌1동%20커스텀커피DMC
이번에 찾을 키워드 : 114 / 115 행 남가좌1동%20속초종합회어시장
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D49943+2595139]
	Ordinal0 [0x00CDC9F1+2148849]
	Ordinal0 [0x00BD4528+1066280]
	Ordinal0 [0x00C00FD4+1249236]
	Ordinal0 [0x00C011CB+1249739]
	Ordinal0 [0x00C2D812+1431570]
	Ordinal0 [0x00C1BA34+1358388]
	Ordinal0 [0x00C2BAF2+1424114]
	Ordinal0 [0x00C1B806+1357830]
	Ordinal0 [0x00BF6086+1204358]
	Ordinal0 [0x00BF6F96+1208214]
	GetHandleVerifier [0x00EEB232+1658114]
	GetHandleVerifier [0x00FA312C+2411516]
	GetHandleVerifier [0x00DDF261+560433]
	GetHandleVerifier [0x00DDE366+556598]
	Ordinal0 [0x00CE286B+2173035]
	Ordinal0 [0x00CE75F8+2192888]
	Ord

<ipython-input-3-3740e2e0aafa>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_map_url'] = "https://m.place.naver.com/restaurant/" + dong_list[0]['naver_map_url']


In [4]:
naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
chromedriver = 'C:/Users/ChaeWon/crawling/chromedriver.exe' 

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(dong_list[0]['naver_map_url']):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(2)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text) #리뷰 내용

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')
 
driver.quit()
sub_driver.quit()


dong_list[0]['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
dong_list[0]['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
dong_list[0]['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
dong_list[0]['naver_blog_review_qty'] = blog_review_qty_list
dong_list[0]['naver_blog_review'] = blog_review_list

<ipython-input-4-48731894fa79>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
<ipython-input-4-48731894fa79>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  sub_driver = webdriver.Chrome(chromedriver)
<ipython-input-4-48731894fa79>:26: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text
<ipython-input-4-48731894fa79>:29: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text


0행 문제가 발생
1행 문제가 발생
2행 문제가 발생
3행 문제가 발생
4행 문제가 발생
5행 문제가 발생
6행 문제가 발생
7행 문제가 발생
8행 문제가 발생
9행 문제가 발생
10행 문제가 발생
11행 문제가 발생
12행 문제가 발생
13행 문제가 발생
14행 문제가 발생
15행 문제가 발생
16행 문제가 발생
17행 문제가 발생
18행 문제가 발생
19행 문제가 발생
20행 문제가 발생
21행 문제가 발생
22행 문제가 발생
23행 문제가 발생
24행 문제가 발생
25행 문제가 발생
26행 문제가 발생
27행 문제가 발생
28행 문제가 발생
29행 문제가 발생
30행 문제가 발생
31행 문제가 발생
32행 문제가 발생
33행 문제가 발생
34행 문제가 발생
35행 문제가 발생
36행 문제가 발생
37행 문제가 발생
38행 문제가 발생
39행 문제가 발생
40행 문제가 발생
41행 문제가 발생
42행 문제가 발생
43행 문제가 발생
44행 문제가 발생
45행 문제가 발생
46행 문제가 발생
47행 문제가 발생
48행 문제가 발생
49행 문제가 발생
50행 문제가 발생
51행 문제가 발생
52행 문제가 발생
53행 문제가 발생
54행 문제가 발생
55행 문제가 발생
56행 문제가 발생
57행 문제가 발생
58행 문제가 발생
59행 문제가 발생
60행 문제가 발생
61행 문제가 발생
62행 문제가 발생
63행 문제가 발생
64행 문제가 발생
65행 문제가 발생
66행 문제가 발생
67행 문제가 발생
68행 문제가 발생
69행 문제가 발생
70행 문제가 발생
71행 문제가 발생
72행 문제가 발생
73행 문제가 발생
74행 문제가 발생
75행 문제가 발생
76행 문제가 발생
77행 문제가 발생
78행 문제가 발생
79행 문제가 발생
80행 문제가 발생
81행 문제가 발생
82행 문제가 발생
83행 문제가 발생
84행 문제가 발생
85행 문제가 발생
86행 문제가 발생
87행 문제가 발생
88행 문제가 발생
89행 문제가 발생
90행 문제가 발생
91행 문제가 발

<ipython-input-4-48731894fa79>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
<ipython-input-4-48731894fa79>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
<ipython-input-4-48731894fa79>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [5]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
dong_list[0] = dong_list[0].loc[~(dong_list[0]['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
dong_list[0][['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = dong_list[0][['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)

In [10]:
dong_list

[Empty DataFrame
 Columns: [상호명, 상권업종대분류명, 상권업종소분류명, 시도명, 시군구명, 행정동명, 도로명주소, 경도, 위도, naver_keyword, naver_map_url, naver_store_type, naver_star_point, naver_star_point_qty, naver_blog_review_qty, naver_blog_review]
 Index: [],
                상호명 상권업종대분류명     상권업종소분류명    시도명  시군구명   행정동명  \
 2420       울랄라포장마차       음식     소주방/포장마차  서울특별시  서대문구  남가좌2동   
 4039      맥도날드명지대점       음식        패스트푸드  서울특별시  서대문구  남가좌2동   
 5378            봄날       음식    한식/백반/한정식  서울특별시  서대문구  남가좌2동   
 15050            필       음식    한식/백반/한정식  서울특별시  서대문구  남가좌2동   
 15741          고기집       음식    한식/백반/한정식  서울특별시  서대문구  남가좌2동   
 ...            ...      ...          ...    ...   ...    ...   
 306868        카페이엘       음식  커피전문점/카페/다방  서울특별시  서대문구  남가좌2동   
 308518       오빠네초밥       음식     음식점-초밥전문  서울특별시  서대문구  남가좌2동   
 308891       엠마테이블       음식       퓨전음식전문  서울특별시  서대문구  남가좌2동   
 309015       프랭크버거       음식        패스트푸드  서울특별시  서대문구  남가좌2동   
 310251  제주흑돼지투접시족발       음식      족발/보쌈전문  서울특별시  서대문구  남가

In [11]:
blog_review_list

['null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null',
 'null']